# STEPS
1. Import libraries
2. Import the dataset
3. Data Analysis - DE, DM, DC, DV, EDA
4. [OPTIONAL] Hyper parameter Tuning
5. Feature Engineering - Encoders, Feature Scaling
6. Split the data into 2 sets using the CV
7. Modek Selection- KNN
8. Training the model
9. Test the model
10. Performance- Confusion Metric


* Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import json
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score

* Import the dataset

Credits: https://drive.google.com/file/d/1nWdt8_Hh-M35N4GwRixPmb3AsHNQVZKQ/view?usp=sharing

Movies: https://drive.google.com/file/d/1HxZXAkPajH40pChWvpAseOi71VW0k3wS/view?usp=sharing

In [ ]:
#2 Datasets---> movies, ratings(credits)

In [ ]:
# Importing the dataset
movies=pd.read_csv('/content/drive/MyDrive/02.04.2024/tmdb_5000_movies.csv')
credits= pd.read_csv('/content/drive/MyDrive/02.04.2024/tmdb_5000_credits.csv')

* Data Analysis - DE, DM, DC, DV, EDA

In [ ]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [ ]:
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [ ]:
# id, movie_id
# original_title, title
# common dta column

In [ ]:
#target --> vote_average

In [ ]:
movies.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [ ]:
credits.describe()

,movie_id
count,4803.000000
mean,57165.484281
std,88694.614033
min,5.000000
25%,9014.500000
50%,14629.000000
75%,58610.500000
max,459488.000000


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
# missing
movies.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [ ]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [ ]:
movies.genres[0]
#the name key's value is genre

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
type(movies.genres[0])

str

In [ ]:
# #  Convert this
# #  [{"id": 28, "name": "Action"},
#     {"id": 12, "name": "Adventure"},
#     {"id": 14, "name": "Fantasy"},
#     {"id": 878, "name": "Science Fiction"}]
# #  to
# #  this
# #  [Action, Adventure, Fantasy, Science Fiction]
# #this is the required simple genre details

#note: that one is not in a list of dict, its on the whole a string:str

In [ ]:
#convert string to JSON-->search in google
#got a function called-->eval

In [ ]:
type(movies.genres[0])

str

In [ ]:
movies.genres[0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
a= eval(movies.genres[0])

In [ ]:
type(a)

list

In [ ]:
a

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [ ]:
a[0]

{'id': 28, 'name': 'Action'}

In [ ]:
a[0]['name']

'Action'

In [ ]:
#Backtracking

In [ ]:
for i in range(len(a)):
  print(a[i]['name'])

Action
Adventure
Fantasy
Science Fiction


In [ ]:
# movies.genres[0]=[a[i]['name'] for i in range(len(a))]

In [ ]:
#for 1
#movies.genres[0]=[eval(movies.genres[0])[i]['name'] for i in range(len(eval(movies.genres[0])))]

*eval

In [ ]:
# for all
# for j in movies.genres:
#   asd=[eval(movies.genres[0])[i]['name'] for i in range(len(eval(movies.genres[0])))]
#################################################################################################(NOTE IT)
#j-->element
#k-->for iteration value
for j,k in zip(movies.genres, range(len(movies.genres))):
  a=eval(j)
  movies.genres[k]=[a[i]['name'] for i in range(len(a))]

<ipython-input-34-da23ab6c44aa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.genres[k]=[a[i]['name'] for i in range(len(a))]


In [ ]:
movies.genres[0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [ ]:
movies.genres[1]

['Adventure', 'Fantasy', 'Action']

In [ ]:
movies.genres

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [ ]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
#do the same for the columns:-
#1. keywords
#2. production_companies
#3. production_countries
#4. spoken_languages

In [ ]:
movies.keywords

0       [{"id": 1463, "name": "culture clash"}, {"id":...
1       [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2       [{"id": 470, "name": "spy"}, {"id": 818, "name...
3       [{"id": 849, "name": "dc comics"}, {"id": 853,...
4       [{"id": 818, "name": "based on novel"}, {"id":...
                              ...                        
4798    [{"id": 5616, "name": "united states\u2013mexi...
4799                                                   []
4800    [{"id": 248, "name": "date"}, {"id": 699, "nam...
4801                                                   []
4802    [{"id": 1523, "name": "obsession"}, {"id": 224...
Name: keywords, Length: 4803, dtype: object

In [ ]:
#keywords
for j,k in zip(movies.keywords, range(len(movies.keywords))):
  a=eval(j)
  movies.keywords[k]=[a[i]['name'] for i in range(len(a))]

<ipython-input-41-bce5504bbd30>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.keywords[k]=[a[i]['name'] for i in range(len(a))]


In [ ]:
movies.production_companies

0       [{"name": "Ingenious Film Partners", "id": 289...
1       [{"name": "Walt Disney Pictures", "id": 2}, {"...
2       [{"name": "Columbia Pictures", "id": 5}, {"nam...
3       [{"name": "Legendary Pictures", "id": 923}, {"...
4             [{"name": "Walt Disney Pictures", "id": 2}]
                              ...                        
4798             [{"name": "Columbia Pictures", "id": 5}]
4799                                                   []
4800    [{"name": "Front Street Pictures", "id": 3958}...
4801                                                   []
4802    [{"name": "rusty bear entertainment", "id": 87...
Name: production_companies, Length: 4803, dtype: object

In [ ]:
#keywords
for j,k in zip(movies.production_companies, range(len(movies.production_companies))):
  a=eval(j)
  movies.production_companies[k]=[a[i]['name'] for i in range(len(a))]

<ipython-input-43-8b18bf676cd6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.production_companies[k]=[a[i]['name'] for i in range(len(a))]


In [ ]:
movies.production_countries

0       [{"iso_3166_1": "US", "name": "United States o...
1       [{"iso_3166_1": "US", "name": "United States o...
2       [{"iso_3166_1": "GB", "name": "United Kingdom"...
3       [{"iso_3166_1": "US", "name": "United States o...
4       [{"iso_3166_1": "US", "name": "United States o...
                              ...                        
4798    [{"iso_3166_1": "MX", "name": "Mexico"}, {"iso...
4799                                                   []
4800    [{"iso_3166_1": "US", "name": "United States o...
4801    [{"iso_3166_1": "US", "name": "United States o...
4802    [{"iso_3166_1": "US", "name": "United States o...
Name: production_countries, Length: 4803, dtype: object

In [ ]:
#production_countries
for j,k in zip(movies.production_countries, range(len(movies.production_countries))):
  a=eval(j)
  movies.production_countries[k]=[a[i]['name'] for i in range(len(a))]

<ipython-input-45-1d1e78fc8157>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.production_countries[k]=[a[i]['name'] for i in range(len(a))]


In [ ]:
movies.spoken_languages

0       [{"iso_639_1": "en", "name": "English"}, {"iso...
1                [{"iso_639_1": "en", "name": "English"}]
2       [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...
3                [{"iso_639_1": "en", "name": "English"}]
4                [{"iso_639_1": "en", "name": "English"}]
                              ...                        
4798        [{"iso_639_1": "es", "name": "Espa\u00f1ol"}]
4799                                                   []
4800             [{"iso_639_1": "en", "name": "English"}]
4801             [{"iso_639_1": "en", "name": "English"}]
4802             [{"iso_639_1": "en", "name": "English"}]
Name: spoken_languages, Length: 4803, dtype: object

In [ ]:
#spoken_languages
for j,k in zip(movies.spoken_languages, range(len(movies.spoken_languages))):
  a=eval(j)
  movies.spoken_languages[k]=[a[i]['name'] for i in range(len(a))]

<ipython-input-47-10db63709afe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.spoken_languages[k]=[a[i]['name'] for i in range(len(a))]


In [ ]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
movies.iloc[140]

budget                                                          150000000
genres                                          [Action, Drama, Thriller]
homepage                                                              NaN
id                                                                 117251
keywords                [usa president, conspiracy, secret service, th...
original_language                                                      en
original_title                                           White House Down
overview                Capitol Policeman John Cale has just been deni...
popularity                                                      39.004588
production_companies    [Columbia Pictures, Centropolis Entertainment,...
production_countries                           [United States of America]
release_date                                                   2013-06-27
revenue                                                         205366737
runtime                               

In [ ]:
movies.isnull().sum ()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [ ]:
movies.shape

(4803, 20)

In [ ]:
# homepage--> has 3091/4803 as null values, can't fill, so just eliminate that entire col
# release_date -->1/4803 as null, can't predict a release date, so remove that row alone
# runtime -->2/4803 null values, can predict
#       see the runtime, if has more outliers, then use mediam
#       if has less outliers, then use mean
#tagline-->844/4803
#       see an example tagline of row 140..'It will start like any other day.'
#       its not in a numerical form to predict
#       these taglines cannot be predicted, so drop this column

In [ ]:
#hompage-eliminate full column
movies.drop('homepage',axis=1, inplace=True)

In [ ]:
#tagline-eliminate full column
movies.drop('tagline',axis=1, inplace=True)

In [ ]:
movies[movies.runtime.isnull()]
#The 2 rows in which 'runtime' is null, in the same rows, 'overview' is also null

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
2656,15000000,[Drama],370980,"[pope, biography]",it,Chiamatemi Francesco - Il Papa della gente,NaN,0.738646,[Taodue Film],[Italy],2015-12-03,0,NaN,[Español],Released,Chiamatemi Francesco - Il Papa della gente,7.3,12
4140,2,[Documentary],459488,"[music, actors, legendary perfomer, classic ho...",en,"To Be Frank, Sinatra at 100",NaN,0.050625,[Eyeline Entertainment],[United Kingdom],2015-12-12,0,NaN,[],Released,"To Be Frank, Sinatra at 100",0.0,0


In [ ]:
#runtime, remove the 2 null value having rows alone
#overview, remove the 3 null value having rows alone
#release_date,remove the 3 null value having rows alone
movies.dropna(inplace=True)
#we use dropna at last because, it drops all rows with null values,
#so use it at last finally

In [ ]:
movies.isnull().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
title                   0
vote_average            0
vote_count              0
dtype: int64

In [ ]:
movies.shape

(4799, 18)

In [ ]:
movies.head(1)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Released,Avatar,7.2,11800


In [ ]:
#release_date is in this form--2009-12-10
#convert--
#release_date->release_day, release_month,  release_year

In [ ]:
df=movies.copy()

In [ ]:
# Idid
#df[['release_year', 'release_month', 'release_day']] = df['release_date'].str.split('-', expand=True)

In [ ]:
df.head(1)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Released,Avatar,7.2,11800


*date->year,month,day

In [ ]:
#Sir did
movies['Year'] = pd.DatetimeIndex(movies.release_date).year
movies['Month'] = pd.DatetimeIndex(movies.release_date).month
movies['Day'] = pd.DatetimeIndex(movies.release_date).day

In [ ]:
movies.head(1)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,Year,Month,Day
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",...,2787965087,162.0,"[English, Español]",Released,Avatar,7.2,11800,2009,12,10


In [ ]:
 #now drp the release_date
 movies.drop('release_date',axis=1, inplace=True)

In [ ]:
#####################################################################

In [ ]:
#CREDITS DATASET

In [ ]:
#####################################################################

In [ ]:
credits.isnull().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64

In [ ]:
#no need to clean

In [ ]:
######################################################

In [ ]:
#merge

In [ ]:
#######################################################

In [ ]:
movies.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'revenue', 'runtime', 'spoken_languages',
       'status', 'title', 'vote_average', 'vote_count', 'Year', 'Month',
       'Day'],
      dtype='object')

In [ ]:
credits.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [ ]:
movies.shape

(4799, 20)

In [ ]:
credits.shape

(4803, 4)

In [ ]:
#merge with
#actually its best if we merged before, because we dont know what rows we dropped, cant drop the same in the second dataset before joining
#but now, as we came this far, we are doing

In [ ]:
movies=pd.merge(movies, credits, left_on='id', right_on='movie_id')
##this is also okay, because, it meges only for the rows where 'id' and 'movied_id' matches
##this is good, this is good

In [ ]:

movies.isnull().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
title_x                 0
vote_average            0
vote_count              0
Year                    0
Month                   0
Day                     0
movie_id                0
title_y                 0
cast                    0
crew                    0
dtype: int64

In [ ]:
movies.head(1)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,title_x,vote_average,vote_count,Year,Month,Day,movie_id,title_y,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",...,Avatar,7.2,11800,2009,12,10,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
movies.cast[0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [ ]:
movies.crew[0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [ ]:
#usefull things in movies.cast[0]
#1. name(list of names of cast)(take where key is 'name')
#2.

#usefull things in movies.crew[0]
#1. name(list of names of crew)(take where key is 'name')
#2. department(list of department of crew)(take where key is 'department')
#3. Director
#eg-- "job": "Director", "name": "James Cameron"},
# if key: is job, value: is director, then get key: name, value-James(need this)
#just 1 name


#note: its in type: str...use eval

In [ ]:
#movies.cast
for j,k in zip(movies.cast, range(len(movies.cast))):
  a=eval(j)
  movies.cast[k]=[a[i]['name'] for i in range(len(a))]

<ipython-input-86-5411690f41d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.cast[k]=[a[i]['name'] for i in range(len(a))]


In [ ]:
#you cant use the same code for the crew, if you run it for 'name'
#then you'll lose the department, director all these
# so create another columns and then do+
#1st do for department, director,
#finally do for names

In [ ]:
m=movies.copy()

In [ ]:
# 1 for loop-- consume too much time
#use 'numba' library
#it will speedup

In [ ]:
!pip install numba

In [ ]:
from numba import jit

*In Pandas there is a functipn 'apply'
you an define a logic of cleaning in that function, then reuse it

In [ ]:
#crew-name
a1=[]
for j,k in zip(m.crew, range(len(m.crew))):
  a=eval(j)
  a1.append([a[i]['department'] for i in range(len(a))])


In [ ]:
a1

[['Editing',
  'Art',
  'Sound',
  'Sound',
  'Production',
  'Sound',
  'Directing',
  'Writing',
  'Editing',
  'Production',
  'Writing',
  'Art',
  'Visual Effects',
  'Production',
  'Art',
  'Sound',
  'Sound',
  'Sound',
  'Sound',
  'Costume & Make-Up',
  'Production',
  'Art',
  'Art',
  'Art',
  'Art',
  'Production',
  'Costume & Make-Up',
  'Camera',
  'Art',
  'Crew',
  'Costume & Make-Up',
  'Costume & Make-Up',
  'Camera',
  'Art',
  'Visual Effects',
  'Crew',
  'Editing',
  'Art',
  'Camera',
  'Crew',
  'Visual Effects',
  'Crew',
  'Art',
  'Art',
  'Art',
  'Crew',
  'Crew',
  'Visual Effects',
  'Art',
  'Art',
  'Production',
  'Art',
  'Costume & Make-Up',
  'Camera',
  'Costume & Make-Up',
  'Visual Effects',
  'Visual Effects',
  'Crew',
  'Visual Effects',
  'Costume & Make-Up',
  'Costume & Make-Up',
  'Visual Effects',
  'Art',
  'Costume & Make-Up',
  'Costume & Make-Up',
  'Costume & Make-Up',
  'Art',
  'Art',
  'Editing',
  'Art',
  'Crew',
  'Crew',
  '

In [ ]:
len(a1)

4799

In [ ]:
#crew-jobs-director----that name
c1=[]
for j,k in zip(m.crew, range(len(m.crew))):
  a=eval(j)
  c=[ a[i]["name"] for i in range(len(a)) if a[i]['job']=="Director" ]
  c1.append(c)


In [ ]:
c1

[['James Cameron'],
 ['Gore Verbinski'],
 ['Sam Mendes'],
 ['Christopher Nolan'],
 ['Andrew Stanton'],
 ['Sam Raimi'],
 ['Byron Howard', 'Nathan Greno'],
 ['Joss Whedon'],
 ['David Yates'],
 ['Zack Snyder'],
 ['Bryan Singer'],
 ['Marc Forster'],
 ['Gore Verbinski'],
 ['Gore Verbinski'],
 ['Zack Snyder'],
 ['Andrew Adamson'],
 ['Joss Whedon'],
 ['Rob Marshall'],
 ['Barry Sonnenfeld'],
 ['Peter Jackson'],
 ['Marc Webb'],
 ['Ridley Scott'],
 ['Peter Jackson'],
 ['Chris Weitz'],
 ['Peter Jackson'],
 ['James Cameron'],
 ['Anthony Russo', 'Joe Russo'],
 ['Peter Berg'],
 ['Colin Trevorrow'],
 ['Sam Mendes'],
 ['Sam Raimi'],
 ['Shane Black'],
 ['Tim Burton'],
 ['Brett Ratner'],
 ['Dan Scanlon'],
 ['Michael Bay'],
 ['Michael Bay'],
 ['Sam Raimi'],
 ['Marc Webb'],
 ['Joseph Kosinski'],
 ['John Lasseter', 'Brad Lewis'],
 ['Martin Campbell'],
 ['Lee Unkrich'],
 ['McG'],
 ['James Wan'],
 ['Marc Forster'],
 ['Bryan Singer'],
 ['J.J. Abrams'],
 ['Bryan Singer'],
 ['Baz Luhrmann'],
 ['Mike Newell'],
 

In [ ]:
# #crew-jobs-director----that name
# '@jit'
# c1=[]
#def run():
  # for j,k in zip(m.crew, range(len(m.crew))):
  #   a=eval(j)
  #   c=[ a[i]["name"] for i in range(len(a)) if a[i]['job']=="Director" ]
  #   c1.append(c)
  #return c1
############################################
# SEE DOCUMENTATION -numba, jit
############################################

In [ ]:
#crew-name
for j,k in zip(m.crew, range(len(m.crew))):
  a=eval(j)
  m.crew[k]=[a[i]['name'] for i in range(len(a))]

<ipython-input-104-c16afd6ad095>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m.crew[k]=[a[i]['name'] for i in range(len(a))]


In [ ]:
movies['Director'] = c1
movies['Department']= a1

In [ ]:
movies.head(1)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,vote_count,Year,Month,Day,movie_id,title_y,cast,crew,Director,Department
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",...,11800,2009,12,10,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[James Cameron],"[Editing, Art, Sound, Sound, Production, Sound..."


In [ ]:
movies.isnull().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
title_x                 0
vote_average            0
vote_count              0
Year                    0
Month                   0
Day                     0
movie_id                0
title_y                 0
cast                    0
crew                    0
Director                0
Department              0
dtype: int64

*Feature selection

In [ ]:
# for movie rating prediction
# need-->[budget, genres, keywords, popularity, revenue,	runtime,	status,	title_x,	vote_count,	Year,	Month,	Day, cast, Director]
# noneed-->[id, original_language,	original_title,	overview,	production_companies, production_countries, crew, Department	]

In [ ]:
# for movie recommendation
# need-->differs
# noneed--> differs
#so export the cleaned data and keep

In [ ]:
movies.status.unique()

array(['Released', 'Post Production', 'Rumored'], dtype=object)

In [ ]:
#Export the dataset

In [ ]:
######################################
#github-ashi36/Movie-Recommendation-Sentimental-Analysis-->clone and do this project, they did use pickle, flask, API(create API), beautiful soup-->fork and dddo

In [ ]:
#do hosting for sure

In [ ]:
#never waste time by creating from scratch

In [ ]:
#in IT world use created codes
#Dot as a flask website

In [ ]:
#otherwise no interviewer will see your collab project

# PROJECT

In [ ]:
########################################
#data.flair.training, it has ml projects on flask, django, use
########################################